In [ ]:
%useLatestDescriptors
%use dataframe

# How to get results from heroku instance?

1. Restore database from heroku into local postgres instance

```bash
make db-dump
```

Heroku cli must by already logged in.

2. Export data from local postgres instance into results.csv with following query:

```sql
with resultats as (
   select r.author, r.comment, p.criterion_id , p.priority, p.comment as priority_comment
   ,min(p.priority) over (partition by r.author) as priority_min
   ,max(p.priority) over (partition by r.author) as priority_max
   from sogisurvey.responses as r
   inner join sogisurvey.priorities as p
   on r.id = p.response_id
)
select author, comment, criterion_id, priority
, 100 * (priority - priority_min) / (priority_max - priority_min) as normalized_priority
, priority_comment
from resultats
``

In [ ]:
val raw = DataFrame.read("./results.csv")
raw.head()

In [ ]:
import com.sogilis.survey.CRITERIA
val responses = raw.add("criterion") { CRITERIA.first { it.id == criterion_id }.title }
responses.head()

In [ ]:
val authors = responses.distinct { author }
val participation_rate = (100.0 * authors.count() / 31).toInt()
val responsesWithComment = responses
    .filter { comment?.isNotBlank() ?: false }
    .distinct { author and comment }
val html = buildString {
    append("<b>${authors.count()}</b> personnes ont répondu (aux de participation: ${participation_rate}%.")
    append("<ul>")
    responsesWithComment.forEach {
        append("<li><b>$author</b> a commenté : \"$comment\"</li>")
    }
    append("</ul>")
}
DISPLAY(HTML(html))

In [ ]:
%use kandy

In [ ]:
responses.add("color") { if (priority_comment != null) "red" else "default" }.plot {
//responses.plot {
    points {
        x(criterion_id) {
            axis.name = "Critère"
        }
        y(normalized_priority) {
            axis.name = "Importance"
            axis.breaks(listOf(0, 50, 100))
        }
        color("color")
        tooltips(author, priority_comment)
        //symbol = Symbol.CROSS
        alpha = .4
    }
    layout {
        style {
            legend.position = LegendPosition.None
        }
        size = 1200 to 800
    }
}
//.save("results.svg")

In [ ]:
// Required to sort my median (see next cell)
val statsPlot = responses.statBoxplot {
    x(criterion_id)
    y(normalized_priority)
}

In [ ]:
statsPlot.sortBy { Stat.middle }.reverse().plot {
    boxes {
        x(Stat.x)
        yMin(Stat.min)
        lower(Stat.lower)
        middle(Stat.middle)
        upper(Stat.upper)
        yMax(Stat.max)
    }
    layout {
        y.axis.name = "Criteria"
        size = 1200 to 600
    }
}

📣 Voici les résultats du sondage **"c'est quoi un bon projet ?"** auquel 21 personnes ont répondu (taux de participation : **68%**). Pour bien interpréter les résultats, un petit [schéma](https://fr.wikipedia.org/wiki/Bo%C3%AEte_%C3%A0_moustaches#/media/Fichier:Boite_a_moustaches_d4.svg) (issu de la page wikipédia sur les [boîtes à moustaches](https://fr.wikipedia.org/wiki/Bo%C3%AEte_%C3%A0_moustaches)) et quelques rappels de définition sur les valeurs importantes :

* La **médiane** est la valeur qui partage une série de données en deux groupes de même nombre d'éléments (différent de la moyenne donc !)
* Les **quartiles** divisent la série en quatre groupes, donc le 1er quartile et le 3e quartile marquent les **bornes à 25% et 75%** (le 2e quartile à 50% étant la médiane)

2 critères sont prépondérants :
* `🔄 Le client souhaite s'améliorer et est à l'écoute de propositions`
* `🎉 Ambiance avec le client`

Ce qui est important mais variable selon les devs :
* `👫 Pouvoir travailler en équipe Sogilis`
* `🐈 Confort de l'environnement de travail chez le client`
* `🏠 Travail à distance`
* `⛵️ Autonomie pour adresser les besoins exprimés`
* `🏙️ Domaine métier`
* `🏃‍♀️ La mission me fait progresser rapidement`

Ce qui est moins important mais pas pour tout le monde :
* `🧑‍🔧 Contexte technique`
* `📍 Localisation du client`
* `👨🏻‍💻 Contraintes sur l'environnement de dev`
* `🧑‍🍳 Type de gestion de projet`
* `🪠 Technos, langages`
* `💻 Base de code existante`

Ce qui n'est pas important :
* `⏱️ Durée de mission`
* `🔖 Type de contrat`